In [ ]:
import sys

# Константа INF задаёт "бесконечность" — число, большее всех возможных чисел во входной матрице.
INF = sys.maxsize

def hungarian_algorithm(cost_matrix, num_rows, num_cols):
    """
    Реализация Венгерского алгоритма для решения задачи о назначениях за время O(n^3).

    Алгоритм работает для прямоугольной матрицы стоимости cost_matrix размером num_rows x num_cols.
    Его цель — минимизировать суммарную стоимость паросочетания.

    Параметры:
        cost_matrix (list of list of int): Матрица стоимости (стоимости назначения строки к столбцу).
        num_rows (int): Число строк в матрице.
        num_cols (int): Число столбцов в матрице.

    Возвращает:
        list of int: Массив, где результат[i] — это номер столбца, сопоставленный строке i + 1.
    """
    # Потенциалы строк и столбцов, изначально равны нулю.
    # Потенциалы корректируют стоимости и позволяют находить новые достижимые вершины.
    row_potentials = [0] * (num_rows + 1)
    col_potentials = [0] * (num_cols + 1)

    # column_matching[i] хранит индекс строки, связанной с i-м столбцом (или 0, если столбец свободен).
    # Это текущее состояние паросочетания.
    column_matching = [0] * (num_cols + 1)

    # Массив предшественников для восстановления увеличивающей цепочки.
    predecessor = [0] * (num_cols + 1)

    # Внешний цикл: добавляем строки в рассмотрение одну за другой.
    for row in range(1, num_rows + 1):
        # Инициализация текущего состояния.
        # Стартуем с фиктивного нулевого столбца, к которому "привязана" текущая строка.
        column_matching[0] = row
        current_column = 0  # Текущий обрабатываемый столбец.

        # min_values[j] — минимальная скорректированная стоимость перехода в столбец j.
        min_values = [INF] * (num_cols + 1)

        # visited[j] — посещён ли столбец j на текущей итерации.
        visited = [False] * (num_cols + 1)

        while True:
            # Помечаем текущий столбец как посещённый.
            visited[current_column] = True

            # Текущая строка, связанная с текущим столбцом.
            current_row = column_matching[current_column]

            # min_delta — минимальное изменение потенциалов, необходимое для достижения нового столбца.
            min_delta = INF
            next_column = 0

            # Обновляем минимальные значения для всех непосещённых столбцов.
            for col in range(1, num_cols + 1):
                if not visited[col]:
                    # Рассчитываем текущую скорректированную стоимость.
                    current_cost = (
                        cost_matrix[current_row - 1][col - 1]
                        - row_potentials[current_row]
                        - col_potentials[col]
                    )
                    # Если нашли меньшую стоимость, обновляем значение.
                    if current_cost < min_values[col]:
                        min_values[col] = current_cost
                        predecessor[col] = current_column  # Запоминаем предшественника.

                    # Обновляем минимальное значение и соответствующий столбец.
                    if min_values[col] < min_delta:
                        min_delta = min_values[col]
                        next_column = col

            # Пересчёт потенциалов: корректируем значения для строк и столбцов.
            for col in range(num_cols + 1):
                if visited[col]:
                    # Обновляем потенциалы для посещённых строк и столбцов.
                    row_potentials[column_matching[col]] += min_delta
                    col_potentials[col] -= min_delta
                else:
                    # Уменьшаем минимальные значения для непосещённых столбцов.
                    min_values[col] -= min_delta

            # Переход к следующему столбцу.
            current_column = next_column

            # Если нашли свободный столбец, выходим из цикла.
            if column_matching[current_column] == 0:
                break

        # "Раскручиваем" увеличивающую цепочку.
        # Используем массив predecessor для восстановления пути.
        while current_column != 0:
            previous_column = predecessor[current_column]
            column_matching[current_column] = column_matching[previous_column]
            current_column = previous_column

    # Восстановление результата: составляем массив из пар строк и столбцов.
    matching_result = [0] * (num_rows + 1)
    for col in range(1, num_cols + 1):
        if column_matching[col] != 0:
            matching_result[column_matching[col]] = col

    # Возвращаем массив, где matching_result[i] — столбец, сопоставленный строке i.
    return matching_result[1:]  # Пропускаем фиктивный нулевой элемент.

if __name__ == "__main__":
    # Пример входных данных: матрица стоимости.
    cost_matrix = [
        [82, 83, 69, 92],
        [77, 37, 49, 92],
        [11, 69, 5, 86],
        [8, 9, 98, 23],
    ]

    num_rows = len(cost_matrix)
    num_cols = len(cost_matrix[0])

    # Выполняем алгоритм.
    result = hungarian_algorithm(cost_matrix, num_rows, num_cols)

    # Вывод результата.
    print("Оптимальное паросочетание:")
    for row in range(1, num_rows + 1):
        print(f"Строка {row} - Столбец {result[row - 1]}")


Оптимальное паросочетание:
Строка 1 - Столбец 3
Строка 2 - Столбец 2
Строка 3 - Столбец 1
Строка 4 - Столбец 4




# Реализация Венгерского алгоритма

## Описание алгоритма

**Венгерский алгоритм** — это метод решения задачи о назначениях, целью которой является минимизация общей стоимости назначения строк матрицы к столбцам. Например, задача может состоять в распределении работников на задачи с минимальными затратами времени или денег.

Алгоритм работает для прямоугольных матриц \( n \times m \), где \( n \leq m \), и имеет временную сложность \( O(n^3) \) для квадратной матрицы и \( O(n^2 \cdot m) \) для прямоугольной.

---

## Идея алгоритма

1. **Представление задачи в виде двудольного графа.**  
   Каждая строка и столбец матрицы представляются вершинами, а веса рёбер — это стоимости из матрицы.

2. **Паросочетание.**  
   Алгоритм постепенно наращивает паросочетание, пока каждая строка не будет сопоставлена одному столбцу.

3. **Потенциалы.**  
   Для каждой строки и столбца вводятся потенциалы, которые корректируют стоимости переходов. Это позволяет эффективно находить новые пути с минимальными изменениями.

4. **Увеличивающие цепочки.**  
   После нахождения свободного столбца восстанавливается путь через предшествующие вершины, что позволяет обновить паросочетание.

5. **Минимизация стоимости.**  
   Пересчёт потенциалов позволяет находить минимальную величину изменения, необходимую для появления нового достижимого столбца.

---

## Алгоритм

1. **Инициализация.**
   - Потенциалы строк и столбцов устанавливаются в ноль.
   - Паросочетание пустое.

2. **Добавление строки.**
   - Каждая строка последовательно добавляется в рассмотрение.

3. **Поиск увеличивающей цепочки.**
   - Для текущей строки запускается процесс нахождения пути в графе. Если найден свободный столбец, обновляется паросочетание.

4. **Пересчёт потенциалов.**
   - Потенциалы строк и столбцов корректируются, чтобы уменьшить стоимости переходов и обеспечить появление новых достижимых вершин.

5. **Раскрутка пути.**
   - После нахождения свободного столбца восстанавливается путь через предшественников и обновляется паросочетание.

6. **Переход к следующей строке.**
   - Алгоритм повторяется, пока все строки не будут включены в паросочетание.

---

## Асимптотика

- **Квадратные задачи (\( n \times n \)):** \( O(n^3) \).
- **Прямоугольные задачи (\( n \times m \)):** \( O(n^2 \cdot m) \).

---

## Пример задачи

Для матрицы стоимости:

```
82 83 69 92
77 37 49 92
11 69  5 86
 8  9 98 23
```

Алгоритм вернёт результат:  
```
Строка 1 - Столбец 3  
Строка 2 - Столбец 2  
Строка 3 - Столбец 1  
Строка 4 - Столбец 4  
```

---

## Использование

Код реализован на Python. Вы можете передать свою матрицу стоимости в функцию `hungarian_algorithm` для получения результата.  

---

больше информации --- http://e-maxx.ru/algo/assignment_hungary#6